# <center>L2 Computational Physics</center>

## <center>Week 3: Differential Equations I</center>

In [1]:
# usual packages to import
import numpy 
import matplotlib.pyplot as plt
%matplotlib inline

In this notebook, you will generate and plot the decay curve for Iodine-133 analytically and numerically. $^{133}\textrm{I}$ has a half life $t_{1/2}$ of 20.8 hours. This means that half of the nuclei will have decayed after time $t_{1/2}$. Derive the mean lifetime $\tau$ from that information.

In [2]:
# define a function to calculate the mean lifetime from the half life
def meanLifetime(halfLife):
    # YOUR CODE HERE
    t = halfLife / numpy.log(2)
    return t
    
T_HALF = 20.8
TAU = meanLifetime(T_HALF)


Check your average lifetime:

In [3]:
# this test is worth 1 mark
assert numpy.isclose(TAU, 30.0080568505)         

### The Decay Equation

Implement the function `f_rad` such that the differential equation 

$$ \frac{dN}{dt} = f_{rad}(N,t)$$

describes the radioactive decay process.

- *Your function should return values using hours as the time unit.*
- *The function should use the constant* `TAU`.

In [4]:
def f_rad(N, t):
    # YOUR CODE HERE
    f = - N / TAU
    return f

Make sure your function works:

In [5]:
# this test cell is worth 1 mark
assert numpy.isclose(f_rad(1000, 0), -33.324383681)           

Solve this first order, ordinary differential equation analytically. Implement this function below, naming it `analytic`. The function should take an initial number of atoms `N0` at time `t=0`, and a time argument. The function should return nuclei count at the time argument. Make sure the function also works for numpy arrays.

In [6]:
def analytic(N0, t):
    # YOUR CODE HERE
    a = N0 * numpy.exp(-t/TAU)
    return a


Check your answer for a single time:

In [7]:
# this test is worth 1 mark
assert numpy.isclose(analytic(1000, 41.6), 250.0)           

In [8]:
# this test is worth 1 mark
assert numpy.isclose(analytic(1000, numpy.arange(0, 60, 6)), 
                     [1000.        ,  818.77471839,  670.39203948,  548.90005334,
                       449.4254866 ,  367.97822623,  301.29126855,  246.68967356,
                       201.983268  ,  165.37879338]).all()


## Numerically Solving the ODE

We now wish to solve our differential equation numerically. We shall do this using Euler's and RK4 methods.

### Euler's Method

Create a function which takes as its arguments the initial number of atoms, `n0`, the initial time `t0`, the time step, `dt`, and the number of steps to perform, `n_steps`.  This function should return an array of the number of counts at each time step using Euler's method. This array should contain the initial and final values, so the array length should be `n_steps+1` 

In [9]:
def solve_euler(f, n0, t0, dt, n_panels):
    # YOUR CODE HERE
    tf = t0 + dt*n_panels #t final
    t = numpy.linspace(t0, tf, n_panels + 1)
    n = numpy.zeros(len(t))
    n[0] = n0
    for i in range(1, len(t)):
        t_i = t0 + i*dt
        n[i] = n[i-1] + f(n[i-1], t_i)*dt
    return n
    
    
    
    

Try your solution:

In [10]:
# this test is worth 1 mark
assert len(solve_euler(f_rad, 1000, 0, 1, 17)) == 18

In [11]:
# this test is worth 2 marks
assert numpy.isclose(solve_euler(f_rad, 1000, 0, 6, 1), [1000.,  800.05369792]).all()

In [12]:
# this test is worth 2 mark
assert numpy.isclose(solve_euler(f_rad, 1000, 0, 6, 10), [1000.        ,  800.05369792,  640.08591955,  512.10310692,
                                                409.7099844 ,  327.7899881 ,  262.24959212,  209.81375595,
                                                167.86227132,  134.29883091,  107.4462763 ]).all()

### RK 4 method

Implement the RK4 method in the `solve_RK4` function. The arguments are the same as for `solve_euler`.

In [13]:
def solve_RK4(f, n0, t0, dt, nsteps):
    # YOUR CODE HERE
    tf = t0 + dt*nsteps #t final
    t = numpy.linspace(t0, tf, nsteps + 1)
    n = numpy.zeros(len(t))
    n[0] = n0
    for i in range(1, len(t)):
        t_i = t0 + i*dt
        k1 = f(n[i-1], t_i)*dt
        k2 = f(n[i-1] + k1/2, t_i + dt/2)*dt
        k3 = f(n[i-1] + k2/2, t_i + dt/2)*dt
        k4 = f(n[i-1] + k3, t_i + dt)*dt
        k = (k1 + 2*k2 + 2*k3 + k4)/6
        n[i] = n[i-1] + k
    return n
    

In [14]:
# This checks that we return an array of the right length
# this test is worth 1 mark
assert len(solve_RK4(f_rad, 1000, 0, 1, 17)) == 18

In [15]:
# This checks that a single step is working
# this test is worth 2 mark
assert numpy.isclose(solve_RK4(f_rad, 1000,0, 6, 1), [1000.,  818.7773]).all()

In [16]:
# This checks multiple steps
# this test is worth 2 marks
assert numpy.isclose(solve_RK4(f_rad, 1000, 0, 6, 10), [
    1000.,
    818.77729521,  
    670.39625915,  
    548.90523578,
    449.43114428,  
    367.9840167,  
    301.29695787,  
    246.69510822, 
    201.98835345,  
    165.3834777,  
    135.41223655]).all()

## Plotting task

**Task 1: **

Create a plot to show that the RK4 method has an error that scales better with the number of steps than the Euler method. (click on the "+" button to create new cells.)       [task worth 5 marks]


In [19]:
t = 10
n0 = 1000
t0 = 0

Nstart = 1
Nmax = 1000
xamount = 100 #hoeveel x en y values
xsteps = (Nmax - Nstart)//(xamount-1) #stapje tussen elke x value

x = [] # number of steps
for i in range(Nstart, Nmax + 1, xsteps):
    x.append(i)

dt = []
for i in range(0, xamount):
    dt.append(t/x[i])
    
y_Euler = []
for i in range(0, xamount):
    Euler = solve_euler(f_rad, n0, t0, dt[i], x[i])
    y_Euler.append(abs(Euler[len(Euler)-1] - analytic(n0, t)))

y_RK4 = []
for i in range(0, xamount):
    RK4 = solve_RK4(f_rad, n0, t0, dt[i], x[i])
    y_RK4.append(RK4[len(RK4)-1] - analytic(n0, t))

plt.figure()
plt.xlabel("number of steps")
plt.ylabel("error at t=15 with n0=1000")

plt.plot()
plt.yscale('log')
plt.xscale('log')
plt.plot(x, y_Euler, label = 'Euler method')
plt.plot(x, y_RK4, label = 'R4R method')
plt.legend()
plt.show()
